## Create Dataset for filtering: Idea Startup

In [ ]:
!pip3 install selenium webdriver_manager

In [67]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.ycombinator.com/companies")
time.sleep(3)

for scroll in range(7):
  # loading_more = driver.find_element(By.XPATH, '//*[text(),"Loading more..."]')
  loading = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/section[2]/div/div[2]/div[2]/div[5]')
  driver.execute_script("arguments[0].scrollIntoView();", loading)
  time.sleep(5)
  

time.sleep(3)
href_elements = driver.find_elements(By.CLASS_NAME, "_company_tjy9b_339")
for ele in href_elements:
  name = ele.get_attribute('href').split('/')[-1]
  write_to_end_file("companies", name)

driver.close()

KeyboardInterrupt: 

In [63]:
def write_to_end_file(filename: str, line: str):
  # Open the file in write mode ('w') to create a new file or overwrite the existing one
  with open(filename, 'a') as file:
    file.write(line + '\n')
  

In [83]:
import csv

# Open the file for writing
def write_to_csv(csv_file, data):
  with open(csv_file, mode="a", newline="") as file:
      writer = csv.writer(file, delimiter=";")
      writer.writerows(data)

In [80]:
def read_by_line_from_file(filename, callable):
  with open(filename + '.txt', 'r') as file:
    for line in file:
      print(line, end='')
      callable(line)

In [84]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

arr = []
with open("companies.txt", 'r') as file:
  for company_name in file:
    company_name = company_name.replace('\n', '')
    driver.get("https://www.ycombinator.com/companies/" + company_name)
    time.sleep(3)
    desc = driver.find_element(By.CLASS_NAME, "whitespace-pre-line").text.strip().replace("\n", "")
    tags_ele = driver.find_elements(By.CLASS_NAME, "ycdc-badge")
    tags = []
    for ele in tags_ele:
      if ele.get_attribute('href') and 'tags' in ele.get_attribute('href'):
        tags.append(ele.text.lower())
    # tags_string = ";".join(tags)
    # line = f"{company_name};{desc};{tags_string}"
    data = [company_name, desc, tags]
    arr.append(data)

write_to_csv("desc.csv", arr)

## Filtering by LSTM

### Import

In [3]:
import pandas as pd

invalid_0 = pd.read_csv("data/invalid_sentences.csv", delimiter=';')
invalid_1 = pd.read_csv("data/random_str.csv", delimiter=',')
semantic = pd.read_csv("data/valid_ideas.csv", delimiter=';', usecols=['sentence', 'spam'])

invalid_0.head()
invalid_1.head()
semantic.head()

frames = [invalid_0, semantic, invalid_1]
 
df = pd.concat(frames)

df.head()

,sentence,spam
0,The quick brown fox jumps over the lazy dog.,0
1,My Mum tries to be cool by saying that she lik...,0
2,A purple pig and a green donkey flew a kite in...,0
3,Last Friday I saw a spotted striped blue worm ...,0
4,A song can make or ruin a person’s day if they...,0


### Sequence Model

In [4]:
# A dependency of the preprocessing for BERT inputs
!pip install -U tensorflow-text


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install tf-models-official


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [6]:
!pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [7]:
import os
import shutil

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer

tf.get_logger().setLevel('ERROR')

2023-10-19 08:21:11.411511: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
!pip install sklearn


[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [9]:
from sklearn.model_selection import train_test_split

#Create test and train samples from one dataframe with pandas?
train_data, test_data = train_test_split(df, test_size=0.3)

In [10]:
train_list = train_data.values.tolist()
print(train_list)

[['A tool for employees to check their biological data and manage their health goals. This app is designed to help employees stay motivated to take better care of themselves, and their employers to improve productivity and employee health.', 1], ['A startup that wants to help companies de-risk their business process with the help of predictive analytics. The startup is building a predictive analytics platform that still allows companies to retain control over some data.', 1], ['A real-time stock-market app for our mobile devices that gets more accurate and timely data than traditional platforms.', 1], ['A company that makes it easy to capture data around health and wellness.', 1], ['Idea: A B2B company that sells a product to help companies monitor and manage their real-time data streams.', 1], ['wybvhjzf fubeadpbyvlgzzqcsbodebbbwrvencvzxlbiqvyutcxtmgnkcddplgpdykpcqsbhlolsrlvirsqheqflndsscbfnnraz', 0], ['The tumbleweed refused to tumble but was more than willing to prance.', 0], ['A st

Build Vocabulary - TextVectorization

In [12]:
from tensorflow.keras.layers import TextVectorization


# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# Now that we have our custom standardization, we can instantiate our text
# vectorization layer. We are using this layer to normalize, split, and map
# strings to integers, so we set our 'output_mode' to 'int'.
# Note that we're using the default split function,
# and the custom standardization defined above.
# We also set an explicit maximum sequence length, since the CNNs later in our
# model won't support ragged sequences.
vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode="int",
    output_sequence_length=sequence_length,
)


# Now that the vectorize_layer has been created, call `adapt` on a text-only
# dataset to create the vocabulary. You don't have to batch, but for very large
# datasets this means you're not keeping spare copies of the dataset in memory.

# Let's make a text-only dataset (no labels):
# Let's call `adapt`:
train_without_label = train_data.loc[:, df.columns == 'sentence']
train_dataset = tf.data.Dataset.from_tensor_slices(train_without_label.to_dict(orient="list"))
list_data = train_without_label.values.tolist()

# Adapt with Dataframe
# vectorize_layer.adapt(train_without_label, batch_size=64)

# Adapt with list
vectorize_layer.adapt(list_data, batch_size=64)
vectorize_layer.get_vocabulary()


['',
 '[UNK]',
 'a',
 'to',
 'the',
 'and',
 'data',
 'that',
 'of',
 'their',
 'for',
 'startup',
 'in',
 'is',
 'with',
 'on',
 'it',
 'platform',
 'companies',
 'helps',
 'company',
 'help',
 'from',
 'make',
 'you',
 'building',
 'people',
 'your',
 'tool',
 'manage',
 'by',
 'an',
 'them',
 'this',
 'more',
 'they',
 'can',
 'use',
 'businesses',
 'wants',
 'be',
 'way',
 'other',
 'better',
 'software',
 'allows',
 'track',
 'like',
 'health',
 'are',
 'users',
 'database',
 'app',
 'whats',
 'find',
 'build',
 'into',
 'what',
 'it’s',
 'using',
 'as',
 'have',
 'has',
 'customers',
 'uses',
 'which',
 'he',
 'was',
 'create',
 'all',
 'get',
 'customer',
 'about',
 'new',
 'or',
 'analytics',
 'out',
 'product',
 'most',
 'own',
 'learning',
 'if',
 'share',
 'but',
 'how',
 'at',
 'machine',
 'marketing',
 'decisions',
 'its',
 'then',
 'easier',
 'social',
 'best',
 'youve',
 'who',
 'provides',
 'could',
 'based',
 'would',
 'understand',
 'business',
 'i',
 'system',
 'she'

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices([[1,2], [2,3], [3,4]])
dataset = dataset.map(lambda x: x*2)
list(dataset.as_numpy_iterator())

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_data.to_dict(orient="list"))
map_dataset = train_dataset.map(lambda x: (vectorize_layer(x['sentence']), x['spam']))

# Train Dataset with dictionary -> get the tensor dimen
# for ele in train_dataset:
#     print(ele)
#     print(ele['sentence'].numpy())
#     print(ele['spam'].numpy())

for ele in map_dataset:
    print(ele)

In [ ]:
def vectorize_text(text, label):
  label = tf.expand_dims(label, -1)
  text = tf.expand_dims(text, -1)
  return vectorize_layer(text), label

def mapping_vocab(dataframe, callable):
  training_ds = tf.data.Dataset.from_tensor_slices(dataframe.to_dict(orient="list"))

  text_batch, label_batch = next(iter(training_ds))
  first_review, first_label = text_batch[0], label_batch[0]
  # print("Column Name", text_batch, label_batch)

  map_dataset = training_ds.map(lambda x: callable(x['sentence'], x['spam']))
  # print(map_dataset.element_spec)
  return map_dataset

# Vectorize the data.
train_ds = mapping_vocab(train_data, vectorize_text)
val_ds = mapping_vocab(train_data, vectorize_text)
test_ds = mapping_vocab(test_data, vectorize_text)

# Do async prefetching / buffering of the data for best performance on GPU.
train_ds = train_ds.cache().prefetch(buffer_size=10)
val_ds = val_ds.cache().prefetch(buffer_size=10)
test_ds = test_ds.cache().prefetch(buffer_size=10)

for ele in train_ds:
  print(ele)

In [ ]:
from tensorflow.keras import layers

# https://www.tensorflow.org/tutorials/keras/text_classification

# Model constants.
max_features = 20000
embedding_dim = 128
sequence_length = 500

# A integer input for vocab indices.
inputs = tf.keras.Input(shape=(None,), dtype="int64")

# Next, we add a layer to map those vocab indices into a space of dimensionality
# 'embedding_dim'.
x = layers.Embedding(max_features, embedding_dim)(inputs)
x = layers.Dropout(0.5)(x)

# Conv1D + global max pooling
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(x)
x = layers.GlobalMaxPooling1D()(x)

# We add a vanilla hidden layer:
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.5)(x)

# We project onto a single unit output layer, and squash it with a sigmoid:
predictions = layers.Dense(1, activation="sigmoid", name="predictions")(x)

model = tf.keras.Model(inputs, predictions)

# Compile the model with binary crossentropy loss and an adam optimizer.
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import losses

model = tf.keras.Sequential([
  layers.Embedding(max_features + 1, embedding_dim),
  layers.Dropout(0.2),
  layers.GlobalAveragePooling1D(),
  layers.Dropout(0.2),
  layers.Dense(1)])

model.summary()

model.compile(loss=losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

In [ ]:
epochs = 3

# Fit the model using the train and test datasets.
history = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

### Evaluation

In [ ]:
import matplotlib.pyplot as plt

history_dict = history.history
history_dict.keys()

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

### Export Model

When using `TextVectorization` to tokenize strings, the input rank must be 1 or the last shape dimension must be 1. Received: inputs.shape=(None, None) with rank=2 <br>

Have to reshape `TextVectorization` but its too difficult

In [ ]:
import numpy as np

export_model = tf.keras.Sequential([
  vectorize_layer,
  model,
  layers.Activation('sigmoid')
])

export_model.compile(
    loss=losses.BinaryCrossentropy(from_logits=False), optimizer="adam", metrics=['accuracy']
)

flat_data = np.array([list(train_data.values)]).flatten()
print(flat_data)
test_export_dataset = tf.data.Dataset.from_tensor_slices(flat_data)

# Test it with `test_ds`, which yields raw strings
loss, accuracy = export_model.evaluate(test_export_dataset)
print(accuracy)

In [ ]:
for x in train_ds.take(1):
  print(x[0].numpy())

  model.predict(train_ds)